# SMuRF basic checks

## Changes
[Describe new version changes and why they were implemented]

## Start server
Navigate to the release directory and start the server with the `run.sh` script.

## Setup system

### Imports

In [ ]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as widgets
import os
import numpy as np

import pysmurf.client

### Get a SmurfControl instance

In [ ]:
slot = 4
epics_prefix = f"smurf_server_s{slot}"
config_file = os.path.abspath("/usr/local/src/pysmurf/cfg_files/rflab/experiment_rflab_2xLB_backplane.cfg")
shelf_man = "shm-smrf-sp01"

In [ ]:
S = pysmurf.client.SmurfControl(
    epics_root=epics_prefix,
    cfg_file=config_file,
    setup=False,
    make_logfile=False,
    shelf_manager=shelf_man
)

What versions of everything are we running?

In [ ]:
pysmurf.__version__

In [ ]:
S._caget(f'{epics_prefix}:AMCc:RogueVersion', as_string=True)

In [ ]:
S._caget(f'{epics_prefix}:AMCc:SmurfApplication:SmurfVersion', as_string=True)

In [ ]:
S._caget(f'{epics_prefix}:AMCc:SmurfApplication:StartupArguments', as_string=True)

In [ ]:
S.get_fpga_git_hash_short()

In [ ]:
bands = []
for i in S.get_enabled_bays():
    bands += [4 * i + j for j in range(4)]

In [ ]:
# convenience function for displaying the output of a given function in tabs per band
def do_tabs(func, *args, bands=bands, **kwargs):
    # create a tab for each band
    tab_outputs = [widgets.Output() for b in bands]
    for i, band in enumerate(bands):
        with tab_outputs[i]:
            func(band, *args, **kwargs)

    # display as tabs
    tab = widgets.Tab(children=tab_outputs)
    for i, band in enumerate(bands):
        tab.set_title(i, f"Band {band}")
    display(tab)

### Run setup
Check that the output contains no errors and the function returns `True`

In [ ]:
assert S.setup(force_configure=True)

### If available, using timing slot

In [ ]:
S.set_timing_mode('backplane')

### Check JESD is locked
(JESD204 is a standard interface for communication links to the FPGA.)

In [ ]:
for i in S.get_enabled_bays():
    print(f"JESD in bay {i}:", S.check_jesd(i))

## Loopback tests
For these tests, no cold resonators are required, and the system should be configured in loopback.

### Bias RF amps for B33 DR RF chain #1

**TODO:** Should we have alternative parameters for the lab 1 setup?

In [ ]:
S.get_amplifier_biases()

In [ ]:
#adjust gate until hemt_Id is 4mA if needed.  
S.set_amp_gate_voltage('hemt1',0.395,override=True)
S.set_amp_drain_voltage('hemt1',0.5)

#50K is B&Z amp that doesn't have a gate.
S.set_amp_drain_voltage('50k1',5.0)

In [ ]:
S.get_amplifier_biases()

Gate voltage corresponding to 4mA is similar to what it usually is.

### Estimate phase delay

Measure the delay in the cable and digital signal processing by fitting for a linear phase trend in the full band response.

The phase plots should look linear in frequency, and the delays should be around 8.5 us for the cable and 0.8 us for the DSP.

In [ ]:
do_tabs(S.estimate_phase_delay)

### Full band response
Measure the response within each band by streaming noise and reading it back.

The result should be relatively smooth across frequencies, except where resonators are present on some bands.

In [ ]:
do_tabs(S.full_band_resp, make_plot=True, show_plot=True)

# TODO:
- currently tests transceivers and low-level DSP
- not analog low-freq low-noise stuff (not in loopback)
- data streaming: track on the cable delay or fixed tones:
    - stream for some amount time in loopback
    - may be more of a hardware test - out of scope
- make a list of existing tests and what they do

## SMB/SPB cold resonator tests

**TODO:** There is a lot of information in this section. Should add some descriptions of what to expect/look out for. Or reduce the amount of things to look through.

### Bias RF amps for B33 DR RF chain #1
(can skip if this was done in the loopback section)

In [ ]:
S.get_amplifier_biases()

In [ ]:
#adjust gate until hemt_Id is 4mA if needed.  
S.set_amp_gate_voltage('hemt1',0.395,override=True)
S.set_amp_drain_voltage('hemt1',0.5)

#50K is B&Z amp that doesn't have a gate.
S.set_amp_drain_voltage('50k1',5.0)

In [ ]:
S.get_amplifier_biases()

Gate voltage corresponding to 4mA is similar to what it usually is.

Set and get UC and DC attenuators.  Matching UC=12, DC=12 to keep ADC from saturating during `find_freqs`.

In [ ]:
bands=[0,3]  # these are where the SMB/SPB are connected

uc_atts={}
dc_atts={}

# SMB is on band 0
uc_atts[0]=24
dc_atts[0]=12

# SPB is on band 3
uc_atts[3]=12
dc_atts[3]=0

for band in bands:
    S.set_att_uc(band,uc_atts[band]) # 0.5dB per step
    S.set_att_dc(band,dc_atts[band]) # 0.5dB per step

    print(S.get_att_uc(band,write_log=True))
    print(S.get_att_dc(band,write_log=True))

### Cable phase delay
(can skip if this was done in the loopback section)

In [ ]:
do_tabs(S.estimate_phase_delay, bands=bands)

### Find resonator frequencies (`find_freq`)

First look for their presence in the quick full band reponse.

In [ ]:
do_tabs(S.full_band_resp, bands=bands, make_plot=True, show_plot=True)

Identify the resonator frequencies. You should see dips in the response at the resonator frequencies.

In [ ]:
do_tabs(S.find_freq, bands=bands, tone_power=12, grad_cut=0.01, amp_cut=0.1, make_plot=True, show_plot=True)

Scan narrower range for clarity; first SMB resonators;

In [ ]:
freqs, resps = S.find_freq(band=0, start_freq=10, stop_freq=150, tone_power=12, grad_cut=0.01, amp_cut=0.1, make_plot=True, show_plot=True)

Now the SPB resonators;

In [ ]:
freqs, resps = S.find_freq(band=3, start_freq=-150, stop_freq=25, tone_power=12,grad_cut=0.01, amp_cut=0.1,make_plot=True,show_plot=True)

### Calibrate the resonator circles (`setup_notches`)
Will also set fixed tones off resonance for phase monitoring. Pick these off resonance;

In [ ]:
fixed_tone_freqs={}
fixed_tone_freqs[0]=4250
fixed_tone_freqs[3]=5687.5

fixed_tone_amp=12

Calibrate the resonator circles and plot. Running `run_serial_gradient_descent` and `run_serial_eta_scan` should refine the eta parameter and resonator frequency estimates.

In [ ]:
for band in bands:
    # Tune
    S.setup_notches(band, new_master_assignment=True)
    
    # Could do this here, but don't for now.
    # Drop DC for for optimal S/N
    #S.set_att_dc(band,2)
    
    # Refine tuning
    S.run_serial_gradient_descent(band)
    
    # Add fixed tone to monitor RF phase
    S.set_fixed_tone(fixed_tone_freqs[band], fixed_tone_amp)
    
    # Refine eta calibrations
    S.run_serial_eta_scan(band)
do_tabs(S.plot_tune_summary, bands=bands, eta_scan=True, show_plot=True)

Should turn off bad channels before attempting to track, but I think everything I just found looks pretty reasonable. Some are slightly collided, but we'll see if/how they track.

Make sure to turn off feedback for the fixed tones and make sure it's center frequency is zero.

In [ ]:
# SMB fixed tone at 4250 MHz
S.set_feedback_enable_channel(0,0,0)

# SPB fixed tone at 5687.5 MHz
S.set_feedback_enable_channel(3,103,0)

Set AC flux ramp mode

In [ ]:
S.set_cryo_card_relay_bit(16,0)
bin(S.get_cryo_card_relays())

How many channels are on in each band?

In [ ]:
for band in bands:
    print(len(S.which_on(band)))

Before moving on to tracking, let's make sure neither the ADC or DAC are saturated;

In [ ]:
for band in bands:
    S.check_dac_saturation(band)
    S.check_adc_saturation(band)

Not clipping either the RF DAC or ADC.

### Tracking setup
Moment of truth - does the flux ramp still work? First the SMB;

In [ ]:
band=0
S.tracking_setup(band, channel = S.which_on(band), reset_rate_khz=4, make_plot=True, save_plot=True, show_plot=True, 
                 lms_freq_hz=None, n_phi0=5, meas_flux_ramp_amp=True, fraction_full_scale=0.2,feedback_gain=2048,lms_gain=0,
                 return_data=False, feedback_start_frac=0.02, feedback_end_frac=0.98,nsamp = 2**18)

Yes! Many nice curves. Let's do the same for SPB resonators.

In [ ]:
band=3
S.tracking_setup(band, channel = S.which_on(band), reset_rate_khz=4, make_plot=True, save_plot=True, show_plot=True, 
                 lms_freq_hz=None, n_phi0=5, meas_flux_ramp_amp=True, fraction_full_scale=0.2,feedback_gain=2048,lms_gain=0,
                 return_data=False, feedback_start_frac=0.02, feedback_end_frac=0.98,nsamp = 2**18)

Looks good! Recording the numerical value of the fraction full scale it settled on here for posterity;

In [ ]:
S.get_fraction_full_scale()

### Take noise data
Let's try taking noise on the SMB+SPB together and see where we land. 5 min.

**TODO**: What to look for in these plots?

In [ ]:
S.take_noise_psd(300,show_plot=True)

Here's a hack for looking at individual timestream plots; first one on SMB that's dark;

In [ ]:
#Import library
from IPython.display import Image
# Load image from local storage
plt.ion()

Image(filename = "/data/smurf_data/20240426/1714162930/plots/1714165921_noise_timestream_b0_ch340.png", width = 600, height = 300)

Now the band 0 fixed tone ;

In [ ]:
Image(filename = "/data/smurf_data/20240426/1714162930/plots/1714165921_noise_timestream_b0_ch000.png", width = 600, height = 300)

That's fun. Now one on SPB that probably has a (superconducting) TES on it;

In [ ]:
Image(filename = "/data/smurf_data/20240426/1714162930/plots/1714165921_noise_timestream_b3_ch495.png", width = 600, height = 300)

I think it's working!

Let's compute noise the same way it's done in sodetlib as a reference.

**TODO**: Is the `sotodlib` noise plot useful here?

---